## Connecting to the chinook database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Take a quick look at the list of tables and views that exist in the chinook database.
We need to remember to add %sql at the beginning of each line of SQL code in python.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM employee
LIMIT 3;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [4]:
%%sql
SELECT *
FROM customer
LIMIT 3;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


Lets take a quick look at how many orders one particular customer has made.

In [5]:
%%sql
SELECT c.customer_id, i.invoice_date, ROUND(i.total,2) 'Sale total'
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
WHERE c.customer_id = 1
ORDER BY 2;

Done.


customer_id,invoice_date,Sale total
1,2017-01-26 00:00:00,8.91
1,2017-05-28 00:00:00,5.94
1,2017-11-30 00:00:00,8.91
1,2017-12-14 00:00:00,13.86
1,2018-02-07 00:00:00,5.94
1,2018-02-09 00:00:00,0.99
1,2018-05-04 00:00:00,17.82
1,2018-08-27 00:00:00,10.89
1,2018-12-10 00:00:00,12.87
1,2019-11-05 00:00:00,3.96


## Analysis to support decisions about which new artists should be promoted in the USA
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name|Genre
| :- | :- 
|Regal |Hip-Hop
|Red Tone|Punk
|Meteor and the Girls|Pop
|Slim Jim Bites|Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

To achieve this, we will first of all create a new 'View' in our database, which holds the total tracks purchased of each genre in the USA.  We can then use this View to enable us to calculate the percentage figures for each genre, in addition to the total count.  

In [6]:
%%sql
DROP VIEW IF EXISTS usa_genre_totals;

CREATE VIEW usa_genre_totals AS
    SELECT i.billing_country country, g.name style, SUM(il.quantity) Total_tracks
    FROM invoice i
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE country = "USA"
    GROUP BY  i.billing_country, t.genre_id 
    ORDER BY 3 DESC;

SELECT *
FROM usa_genre_totals;

Done.
Done.
Done.


country,style,Total_tracks
USA,Rock,561
USA,Alternative & Punk,130
USA,Metal,124
USA,R&B/Soul,53
USA,Blues,36
USA,Alternative,35
USA,Latin,22
USA,Pop,22
USA,Hip Hop/Rap,20
USA,Jazz,14


Now we can add the percentage of tracks of each genre sold.

In [7]:
%%sql
SELECT *, ROUND(CAST(Total_tracks AS FLOAT)/1051*100,2) AS 'Genre %'
FROM usa_genre_totals;

Done.


country,style,Total_tracks,Genre %
USA,Rock,561,53.38
USA,Alternative & Punk,130,12.37
USA,Metal,124,11.8
USA,R&B/Soul,53,5.04
USA,Blues,36,3.43
USA,Alternative,35,3.33
USA,Latin,22,2.09
USA,Pop,22,2.09
USA,Hip Hop/Rap,20,1.9
USA,Jazz,14,1.33


So now let's just look at the figures for the four genres we are intrested in; Hip-Hop, Punk, Pop and Blues.

Before we do this, we should note that although Blues and Pop are genres in their own right:
- Punk is included in the 'Alternative & Punk' genre; and
- Hip-Hop is included in the 'Hip Hop/Rap' category.

In [8]:
%%sql
SELECT *, ROUND(CAST(Total_tracks AS FLOAT)/1051*100,2) AS 'Genre %'
FROM usa_genre_totals
WHERE style IN ('Hip Hop/Rap', 'Alternative & Punk', 'Pop', 'Blues');

Done.


country,style,Total_tracks,Genre %
USA,Alternative & Punk,130,12.37
USA,Blues,36,3.43
USA,Pop,22,2.09
USA,Hip Hop/Rap,20,1.9


## Recomendations about which three artists should be promoted in the USA
From the table above we see that of the four artists proposed by the new record label, Chinook should focus on the artists who are cataegorised as being of the Punk, Blues or Pop genres.

Our recomendation is that Chinook should therefore promote the following artists:
#### - Red Tone
#### - Meteor and the Girls; and
#### - Slim Jim Bites.

## Sales Support Agent analysis

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

We will first of all look at the total sales that each Sales Support Agent (SSA) has attained over their full length of service.  If we save this as a 'view' within our database, then we can use it to calculate their sales ratios.

In [9]:
%%sql
DROP VIEW IF EXISTS employee_sales;

CREATE VIEW employee_sales AS
    SELECT e.employee_id, SUM(ts.customer_sales) Agent_Sales, e.title, e.hire_date, CAST (
    JulianDay('2020-12-31 00:00:00') - JulianDay(e.hire_date) As Integer) Days_in_service
    FROM employee e
    LEFT JOIN
        (
        SELECT c.customer_id, SUM(i.total) AS 'customer_sales', c.support_rep_id
        FROM customer c
        LEFT JOIN invoice i ON i.customer_id = c.customer_id
        GROUP BY 1
        ORDER BY 2 DESC
        ) ts ON ts.support_rep_id = e.employee_id
    WHERE e.title = 'Sales Support Agent'
    GROUP BY 1
    ORDER BY 2 DESC;
    
SELECT * FROM employee_sales;

Done.
Done.
Done.


employee_id,Agent_Sales,title,hire_date,Days_in_service
3,1731.51,Sales Support Agent,2017-04-01 00:00:00,1370
4,1584.0,Sales Support Agent,2017-05-03 00:00:00,1338
5,1393.92,Sales Support Agent,2017-10-17 00:00:00,1171


To determine if the 6 months difference in length of service accounts correlates to the difference in their respective sales we will now calculate their sales per day employed performance; using the employee_sales view we saved in the previous line.

In [10]:
%%sql
SELECT employee_id, Agent_Sales, Days_in_service, Agent_Sales/Days_in_service AS sales_rate
FROM employee_sales
ORDER BY 4 DESC;

Done.


employee_id,Agent_Sales,Days_in_service,sales_rate
3,1731.51,1370,1.263875912408759
5,1393.92,1171,1.1903672075149445
4,1584.0,1338,1.1838565022421526


This tells us that the difference in their lengths of service does not account for the difference in the cumulative sales achieved by the three SSA's.

Indeed, SSA 3 has \$0.07 per day higher rate of sales than SSA 5 and \$0.08 more than SSA 4.  This would suggest that SSA 3 is consistently performing better than the other two SSA's, with the other two having relatively comparable performance.

# Sales analysis by country of customer

The next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

We are therefore aiming to write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
For each country, we will include:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [11]:
%%sql
DROP VIEW IF EXISTS summary_table;

CREATE VIEW summary_table AS
    WITH
        country_count AS
            (
            SELECT
                c.country,
                COUNT(c.country) customers
            FROM customer c
            GROUP BY 1
            ),
        country_totals AS
            (
            SELECT
                c.country,
                i.customer_id,
                COUNT(i.invoice_id) no_of_sales,
                SUM(i.total) total_sales
            FROM invoice i
            LEFT JOIN customer c ON c.customer_id = i.customer_id
            GROUP BY 2
            )
    SELECT
        cc.country,
        cc.customers no_of_customers,
        ct.no_of_sales,
        ct.total_sales,
        ct.total_sales/cc.customers average_customer_sale,
        ct.total_sales/ct.no_of_sales average_order_value,
        CASE
            WHEN cc.customers = 1 THEN "Other"
            WHEN cc.customers >1 THEN cc.country
            END AS country_name
    FROM country_count cc
    LEFT JOIN country_sales ct ON ct.country = cc.country
    GROUP BY 1;

DROP VIEW IF EXISTS final_summary;

CREATE VIEW final_summary AS
    SELECT
        country_name country,
        SUM(no_of_customers) customers,
        SUM(no_of_sales) no_of_orders,
        ROUND(SUM(total_sales), 2) total_sales,
        ROUND(AVG(average_order_value), 3) average_order,
        ROUND(AVG(average_customer_sale), 3) customer_lifetime_value 
    FROM summary_table
    GROUP BY 1;

SELECT
    fs.*,
    CASE
        WHEN fs.country = 'Other' THEN 1
        ELSE 0
    END AS sort
FROM final_summary fs
ORDER BY sort ASC, 5 DESC;

Done.
Done.
Done.
Done.
Done.


country,customers,no_of_orders,total_sales,average_order,customer_lifetime_value,sort
Czech Republic,2,30,273.24,9.108,136.62,0
United Kingdom,3,28,245.52,8.769,81.84,0
India,2,21,183.15,8.721,91.575,0
Germany,4,41,334.62,8.161,83.655,0
USA,13,131,1040.49,7.943,80.038,0
France,5,50,389.07,7.781,77.814,0
Canada,8,76,535.59,7.047,66.949,0
Brazil,5,61,427.68,7.011,85.536,0
Portugal,2,29,185.13,6.384,92.565,0
Other,15,147,1094.94,7.445,72.996,1


Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

# Albums or Tracks analysis

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

# NEXT WORK
Going to try and create one table (view) of the tracks purchased in each invoice
Then another table of all of the tracks in each album
Then do the comparing thing using CASE, to identify where all of the tracks in an album were purchased etc.
THEN SEE WHERE WE GO FROM THERE!

In [12]:
%%sql
-- First of we will count the number of invoices

SELECT
    COUNT(i.invoice_id) 
FROM invoice i;

Done.


COUNT(i.invoice_id)
614


In [13]:
%%sql

-- Now let us try and work out how many tracks were purchased with each invoice

SELECT i.invoice_id, il.track_id example_track, t.album_id, COUNT(il.track_id) tracks_purchased
FROM invoice i
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
GROUP BY 1;


Done.


invoice_id,example_track,album_id,tracks_purchased
1,1173,91,16
2,2651,215,10
3,2646,214,2
4,2514,203,8
5,2002,163,17
6,30,5,2
7,42,6,11
8,1560,125,10
9,196,20,9
10,3290,257,2


In [14]:
%%sql

-- Now let us try and work out how many tracks there are in each album

SELECT t.album_id, COUNT(t.track_id) tracks_on_album
FROM track t
GROUP BY 1;

Done.


album_id,tracks_on_album
1,10
2,1
3,3
4,8
5,15
6,13
7,12
8,14
9,8
10,14


In [16]:
%%sql

-- THIS QUERYVSIMPLY IDENTIFIES WHERE THE NUMBER OF TRACKS PURCHASED
-- HAPPENS TO EQUAL THE NUMBER OF TRACKS THAT ARE ON AN ALBUM THAT
-- ONE OF THOSE TRACKS WAS ON

WITH
    purchased AS
        (
        SELECT
            i.invoice_id invoice_id, 
            il.track_id example_track, 
            t.album_id album_id, 
            COUNT(il.track_id) tracks_purchased
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        GROUP BY 1
        ),
    album_tracks AS
        (
        SELECT 
            t.album_id album_id, 
            COUNT(t.track_id) tracks_on_album
        FROM track t
        GROUP BY 1),
    equal_count as
        (
        SELECT 
            p.invoice_id, 
            p.album_id, 
            p.tracks_purchased, 
            at.tracks_on_album
        FROM purchased p
        INNER JOIN album_tracks at ON at.album_id = p.album_id
        WHERE p.tracks_purchased = at.tracks_on_album
        ORDER BY 1)
SELECT COUNT()
FROM equal_count;

Done.


COUNT()
125


In [17]:
%%sql

-- THIS QUERY SIMPLY IDENTIFIES WHERE THE NUMBER OF TRACKS PURCHASED
-- HAPPENS IS NOT EQUAL TO THE NUMBER OF TRACKS THAT ARE ON AN ALBUM THAT
-- ONE OF THOSE TRACKS WAS ON

WITH
    purchased AS
        (
        SELECT
            i.invoice_id invoice_id, 
            il.track_id example_track, 
            t.album_id album_id, 
            COUNT(il.track_id) tracks_purchased
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        GROUP BY 1
        ),
    album_tracks AS
        (
        SELECT 
            t.album_id album_id, 
            COUNT(t.track_id) tracks_on_album
        FROM track t
        GROUP BY 1),
    unequal_count as
        (
        SELECT 
            p.invoice_id, 
            p.album_id, 
            p.tracks_purchased, 
            at.tracks_on_album
        FROM purchased p
        INNER JOIN album_tracks at ON at.album_id = p.album_id
        WHERE p.tracks_purchased != at.tracks_on_album
        ORDER BY 1)
SELECT COUNT()
FROM unequal_count;

Done.


COUNT()
489


# THE NUMBERS ARE NOT RIGHT SO NEED TO GET THIS MORE ACCURATE.

In [27]:
%%sql

-- THIS QUERYVSIMPLY IDENTIFIES WHERE THE NUMBER OF TRACKS PURCHASED
-- HAPPENS TO EQUAL THE NUMBER OF TRACKS THAT ARE ON AN ALBUM THAT
-- ONE OF THOSE TRACKS WAS ON

WITH
    purchased AS
        (
        SELECT
            i.invoice_id invoice_id, 
            il.track_id example_track, 
            t.album_id album_id, 
            COUNT(il.track_id) tracks_purchased
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        GROUP BY 1
        ),
    album_tracks AS
        (
        SELECT 
            t.album_id album_id, 
            COUNT(t.track_id) tracks_on_album
        FROM track t
        GROUP BY 1),
    equal_count as
        (
        SELECT 
            p.invoice_id, 
            p.album_id, 
            p.tracks_purchased, 
            at.tracks_on_album
        FROM purchased p
        INNER JOIN album_tracks at ON at.album_id = p.album_id
        WHERE p.tracks_purchased = at.tracks_on_album
        ORDER BY 1)
SELECT *
FROM equal_count;

Done.


invoice_id,album_id,tracks_purchased,tracks_on_album
1,91,16,16
5,163,17,17
23,1,10,10
24,90,12,12
31,221,20,20
32,6,13,13
44,242,12,12
46,214,11,11
49,15,5,5
52,216,12,12


In [35]:
%%sql
SELECT
    i.invoice_id invoice_id, 
    il.track_id track_ID, 
    t.album_id album_id 
FROM invoice i
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
ORDER BY 1;

Done.


invoice_id,track_ID,album_id
1,1158,91
1,1159,91
1,1160,91
1,1161,91
1,1162,91
1,1163,91
1,1164,91
1,1165,91
1,1166,91
1,1167,91
